# Математические пакеты. Тестовое задание

Выполнила студентка группы 9381 Андрух Ирина.

## Цели работы

1. Индивидуальное творческое соревнование.
2. Самостоятельное освоение одного из важных методов обучения без учителя - обучение без учителя.
3. Практическое применение дискретного преобразования Фурье для извлечения признаков обучения.
4. Использованием библиотек Python для кластеризации музыкальных сигналов.



In [61]:
import librosa
import scipy
from pydub import silence, AudioSegment
from scipy import signal
from scipy.io import wavfile
import numpy as np
import sklearn.cluster
import csv
import matplotlib.pyplot as plt
import IPython.display as ipd
import wave
import more_itertools as mi
import math
import more_itertools as mit


In [62]:
n = 124
files = []
for i in range(1, n+1):
    if i < 10:
        files.append("mp3_test/0" + str(i) + ".mp3")
    else:
        files.append("mp3_test/" + str(i)  + ".mp3")          

In [63]:
signals = []
a = int
offsetwindow = 256
dimension = 256
for i in range(n):
    audiofile = AudioSegment.from_mp3(files[i]).set_channels(1)
    signals.append(np.array(audiofile.get_array_of_samples()))

In [65]:
a_s = []
for signal in signals:
    intervals = list(mit.windowed(signal, fillvalue=0, n=dimension, step=offsetwindow))
    bound = 0
    for i in intervals:
        m = np.max(i)
        if m > bound:
            bound = m
    bound *= 0.2
    intervals = list(filter(lambda i: np.max(i) > bound, intervals))
    intervals = list(map(lambda interval: scipy.fft.fft(interval * np.hamming(len(interval))), intervals))
    amplitude_spectral = [[np.abs(i) for i in interval[:dimension//2]] for interval in intervals]
    a = [0 for i in range(dimension // 2)]
    for i in amplitude_spectral:
        for j in range(dimension // 2):
            a[j] += i[j]
    for i in range(dimension // 2):
        a[i] /= len(intervals)    
    a_s.append(a)

In [66]:
outs = [0 for _ in range(len(a_s))]

def data_distribution(data, centers):
    dim = len(data[0])
    n = len(data)
    k = len(centers)
    clusters = [[] for i in range(k)]
    for i in range(n):
        min_distance = float('inf')
        situable_cluster = -1
        
        for j in range(k):
            distance = 0
            for q in range(dim):
                distance += (data[i][q] - centers[j][q]) ** 2
            distance = distance ** 0.5
            
            if distance < min_distance:
                min_distance = distance
                situable_cluster = j

        clusters[situable_cluster].append(i)
        outs[i]=situable_cluster
    return clusters

def centers_update(data, clusters, centers):
    dim = len(data[0])
    k = len(centers)
    
    for i in range(k): # по k кластерам
        for q in range(dim): # по dim параметрам
            if len(clusters[i]) != 0:
                updated_parameter = 0
                for j in range(len(clusters[i])): 
                    updated_parameter += data[clusters[i][j]][q]
                updated_parameter = updated_parameter / len(clusters[i])
                centers[i][q] = updated_parameter

def clusterization(data, k):
    n = len(data)  
    dim = len(data[0])  

    indexes = []
    for i in range(k):
        r = random.randint(0, k - 1)
        while r in indexes:
            r = random.randint(0, k - 1)
        indexes.append(r)
        
    centers = [copy.deepcopy(data[indexes[i]]) for i in range(k)]
    clusters = data_distribution(data, centers)
    previous_centers = None
        
    while centers != previous_centers:
        previous_centers = copy.deepcopy(centers)
        centers_update(data, clusters, centers)
        clusters = data_distribution(data, centers)        
    print("\nКомпактность:\n")
    compact=0
    centers = [copy.deepcopy(data[indexes[i]]) for i in range(5)]
    for i in range(len(centers)):
        for j in range(len(clusters[i])):
            for q in range(dim):
                compact+=(data[clusters[i][j]][q]-centers[i][q])**2
    compact=compact/5
    print(compact);
    print("\n");
    return clusters
b = clusterization(a_s, 5)
print("\nКластеризация\n")
print(b)
for i in range(len(a_s)):
    for j in range(len(b)):
        if i in b[j]:
            print(j, end=", ")
            


Компактность:

561478005808.8925



Кластеризация

[[12, 35, 36, 37, 38, 39, 42, 43, 44, 45, 46, 47, 52, 55, 63, 64, 69, 71, 73, 74, 75, 76, 77, 78, 81, 83, 95, 105, 107, 123], [1, 4, 7, 10, 27, 41, 53, 54, 56, 57, 58, 60, 61, 62, 66, 79, 82, 85, 86, 88, 90, 92, 94, 103], [13, 15, 20, 21, 22, 93, 108, 117], [0, 2, 3, 5, 6, 8, 9, 11, 14, 16, 17, 24, 25, 26, 28, 30, 31, 33, 34, 59, 65, 67, 97, 98, 99, 100, 101, 102, 106, 110, 111, 112, 113, 114, 115, 116, 118, 119, 120, 121, 122], [18, 19, 23, 29, 32, 40, 48, 49, 50, 51, 68, 70, 72, 80, 84, 87, 89, 91, 96, 104, 109]]
3, 1, 3, 3, 1, 3, 3, 1, 3, 3, 1, 3, 0, 2, 3, 2, 3, 3, 4, 4, 2, 2, 2, 4, 3, 3, 3, 1, 3, 4, 3, 3, 4, 3, 3, 0, 0, 0, 0, 0, 4, 1, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 0, 1, 1, 0, 1, 1, 1, 3, 1, 1, 1, 0, 0, 3, 1, 3, 4, 0, 4, 0, 4, 0, 0, 0, 0, 0, 0, 1, 4, 0, 1, 0, 4, 1, 1, 4, 1, 4, 1, 4, 1, 2, 1, 0, 4, 3, 3, 3, 3, 3, 3, 1, 4, 0, 3, 0, 2, 4, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 0, 

In [67]:
with open('result.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(len(outs)):
        spamwriter.writerow([files[i], outs[i]])